# Europan Scraper and Analyser

In [36]:
import requests
import json
import pandas as pd
import os

projects = os.listdir('projects_archive')



In [47]:
def resize_image(width, height, threshold):
    if (width > threshold) or (height > threshold):
        max_dim = max(width, height)
        if height == max_dim:
            new_width = int((width * threshold) / height)
            new_height = threshold
        if width == max_dim:
            new_height = int((height * threshold) / width)
            new_width = threshold
        return new_width, new_height
    else: return width, height

In [108]:
import colorsys


# Function to convert the RGB string to a tuple of integers
def rgb_to_tuple(rgb_str):
    return tuple(map(int, rgb_str.split(':')))

# Function to convert RGB to HSV
def rgb_to_hsv(rgb_tuple):
    r, g, b = [x / 255.0 for x in rgb_tuple]
    return colorsys.rgb_to_hsv(r, g, b)


def sort_colors_img(colors):
    empty_img = Image.new('RGB', (len(colors)*10, 500), (255, 255, 255))
    
    # plot colors
    x = 0
    for color in colors:
        color = color[0].split(':')
        r, g, b = int(color[0]), int(color[1]), int(color[2])
        empty_img.paste((r, g, b), (x, 0, x+10, 500))
        x += 10
    empty_img
    return empty_img

In [103]:
from PIL import Image

import sys 
# instlal pillow with pip install pillow

for project in projects: 
    try:
        planches = 'projects_archive/' + project + '/merged_planches.png'
        if os.path.exists(planches):
            print(planches)
            img = Image.open(planches)
            img = img.convert('RGB')
            img = img.resize((500, 500))
            colors = img.getcolors(img.size[0]*img.size[1])
            width, height = img.size
            detected_colors = {} # hash-map
            for x in range(0, width):
                for y in range(0, height):
                    # r,g,b value of pixel
                    r, g, b = img.getpixel((x, y))
                    rgb = f'{r}:{g}:{b}'
                    if rgb in detected_colors:
                        detected_colors[rgb] += 1
                    else: 
                        detected_colors[rgb] = 1
            # Sort colors from most common to least common
            detected_colors = sorted(detected_colors.items(), key=lambda x:x[1], reverse=True)
            # Convert the list to include HSV values
            color_tuples_with_hsv = [(rgb_to_tuple(color), rgb_to_hsv(rgb_to_tuple(color)), count) 
                                    for color, count in detected_colors]

            # Sort the colors based on HSV values
            # This sorts primarily by hue, then by saturation, and finally by value
            sorted_colors = sorted(color_tuples_with_hsv, key=lambda x: (x[1][0], x[1][1], x[1][2]))

            # Convert back to the original format
            sorted_detected_colors = [(':'.join(map(str, color)), count) for color, hsv, count in sorted_colors]

            with open('projects_archive/' + project + '/sorted_colors.json', 'w') as f:
                json.dump(sorted_detected_colors, f)
    except Exception as e:
        print(e)
        print('Error with project', project)
        continue
        
        


projects_archive/0/merged_planches.png
projects_archive/1/merged_planches.png
projects_archive/10/merged_planches.png
projects_archive/100/merged_planches.png
projects_archive/101/merged_planches.png
projects_archive/102/merged_planches.png
projects_archive/103/merged_planches.png
projects_archive/104/merged_planches.png
projects_archive/105/merged_planches.png
projects_archive/106/merged_planches.png
projects_archive/107/merged_planches.png
projects_archive/108/merged_planches.png
projects_archive/109/merged_planches.png
projects_archive/11/merged_planches.png
projects_archive/110/merged_planches.png
projects_archive/111/merged_planches.png
projects_archive/112/merged_planches.png
projects_archive/113/merged_planches.png
projects_archive/114/merged_planches.png
projects_archive/115/merged_planches.png
projects_archive/116/merged_planches.png
projects_archive/117/merged_planches.png
projects_archive/118/merged_planches.png
projects_archive/119/merged_planches.png
projects_archive/12/me

In [115]:
import pandas as pd 
data = pd.read_excel(r'C:\Users\jaddh\code_projects\scrapy\europan_competitions_archive.xlsx')

sessions = data['sessionid'].unique()

for session in sessions: 
    sesion_projects = data[data['sessionid'] == session]
    # get the indexes of the projects
    sesion_projects = sesion_projects.index.tolist()
    colors = []
    for project_id in sesion_projects: 
        colors_route = 'projects_archive/' + str(project_id) + '/sorted_colors.json'
        if os.path.exists(colors_route):
            with open(colors_route, 'r') as f:
                colors += json.load(f)
    unique_colors = set([color[0] for color in colors])
    for color in unique_colors: 
        count = 0
        for color_ in colors: 
            if color_[0] == color: 
                count += color_[1]
        colors.append((color, count))
    colors = sorted(colors, key=lambda x: x[1], reverse=True)
    with open('sessions/' + str(session) + '.json', 'w') as f:
        json.dump(colors, f)
    print(len(colors))
    img = sort_colors_img(colors)
    print('sessions/' + str(session) + '.png')
    img = img.resize((1000, 1000))
    print('sessions/' + str(session) + '.png')
    img.save('sessions/' + str(session) + '.png')